In [1]:
import numpy as np

In [190]:
class Expression:
    
    @staticmethod
    def add(a, b):
        return a + b
    
    @staticmethod
    def subtract(a, b):
        return a - b
    
    @staticmethod
    def multiply(a, b):
        return a * b
    
    @staticmethod
    def divide(a, b):
        return a / b
    
    def __init__(self, operation, operands):
        assert operation in (Expression.add, Expression.subtract, Expression.multiply, Expression.divide)
        self.operation = operation
        self.operands = operands
        self.vertical_view = self.get_vertical_view(with_solution=False)
        self.vertical_view_sol = self.get_vertical_view(with_solution=True)
    
    def evaluate(self):
        result = self.operands[0]
        for n in self.operands[1:]:
            result = self.operation(result, n)
        return result
    
    def get_vertical_view(self, with_solution=True):
        str_operands = [str(n) for n in self.operands]
        sig_digit = max((len(n) for n in str_operands))
        
        grid = np.empty((len(self.operands) + 2, max(sig_digit, len(str(self.evaluate())))), dtype=np.str)
        grid[:,:] = ' '
        grid[-2,:] = '_'
        for i in range(len(str_operands)):
            operand = str_operands[i]
            for j in range(len(operand)):
                grid[i,-j-1] = operand[-j-1]
                
        if with_solution:
            solution = str(self.evaluate())
            for j in range(len(solution)):
                grid[-1,-j-1] = solution[-j-1]
        return grid
        
    def get_op_string(self):
        if self.operation is Expression.add:
            return '+'
        if self.operation is Expression.subtract:
            return '-'
        if self.operation is Expression.multiply:
            return '*'
        if self.operation is Expression.divide:
            return '/'
    
    def __repr__(self):
        return self.get_op_string().join((str(n) for n in self.operands)) + '=' + str(self.evaluate())

In [194]:
e = Expression(Expression.add, [12, 4])
print(e)
print(e.vertical_view_sol)

12+4=16
[['1' '2']
 [' ' '4']
 ['_' '_']
 ['1' '6']]


In [265]:
class ExpressionSolver:
    
    def __init__(self, expression):
        self.expression = expression
        view_coords = []
        current_value = 0
    
    def view(self, coord):
        self.view_coords.append(coord)
        return self.expression.vertical_view[coord]
    
    def get_last_view(self):
        return self.expression.vertical_view[view_coords[-1]]
    
class ViewDirection:
    
    UP_EDGE = "UP_EDGE"
    DOWN_EDGE = "DOWN_EDGE"
    LEFT_EDGE = "LEFT_EDGE"
    RIGHT_EDGE = "RIGHT_EDGE"
    UP_RELATIVE = "UP_RELATIVE"
    DOWN_RELATIVE = "DOWN_RELATIVE"
    LEFT_RELATIVE = "LEFT_RELATIVE"
    RIGHT_RELATIVE = "RIGHT_RELATIVE"
    DO_NOTHING = "DO_NOTHING"
    
    def __init__(self, vertical, horizontal):
        assert vertical in (ViewDirection.UP_EDGE,
                           ViewDirection.DOWN_EDGE,
                           ViewDirection.UP_RELATIVE,
                           ViewDirection.DOWN_RELATIVE,
                           ViewDirection.DO_NOTHING)
        assert horizontal in (ViewDirection.LEFT_EDGE,
                           ViewDirection.RIGHT_EDGE,
                           ViewDirection.LEFT_RELATIVE,
                           ViewDirection.RIGHT_RELATIVE,
                           ViewDirection.DO_NOTHING)
        
        self.vertical = vertical
        self.horizontal = horizontal
        
    def get_coord(self, cur_coord, vertical_view_shape):
        vert = None
        horiz = None
        
        if self.vertical == ViewDirection.UP_EDGE:
            vert = 0
        elif self.vertical == ViewDirection.DOWN_EDGE:
            vert = vertical_view_shape[0] - 1
        elif self.vertical == ViewDirection.UP_RELATIVE:
            vert = cur_coord[0] - 1
        elif self.vertical == ViewDirection.DOWN_RELATIVE:
            vert = cur_coord[0] + 1
        elif self.vertical == ViewDirection.DO_NOTHING:
            vert = cur_coord[0]
        else:
            assert False

        if self.horizontal == ViewDirection.LEFT_EDGE:
            horiz = 0
        elif self.horizontal == ViewDirection.RIGHT_EDGE:
            horiz = vertical_view_shape[1] - 1
        elif self.horizontal == ViewDirection.LEFT_RELATIVE:
            horiz = cur_coord[1] - 1
        elif self.horizontal == ViewDirection.RIGHT_RELATIVE:
            horiz = cur_coord[1] + 1
        elif self.horizontal == ViewDirection.DO_NOTHING:
            horiz = cur_coord[1]
        else:
            assert False
            
        return (vert, horiz)
    
    def __repr__(self):
        return str((self.vertical, self.horizontal))
    
    
class AdditionSolver:
    
    def __init__(self, expression):
        assert expression.operation == Expression.add
        self.expression = expression
        self.view_dir_history = []
        self.coord_history = []
        self.write_history = []
        self.current_value = 0
        
    """
    Returns a ViewDirection, next_coord, and a digit to write
    """
    def action(self, view_direction):
        vertical_view = self.expression.vertical_view_sol
        last_coord = self.coord_history[-1] if self.coord_history else None
        write = None
        
        # If beginning of problem, i.e. nothing has been done yet
        # Move to top-right corner
        if view_direction is None:
            next_view_direction = ViewDirection(ViewDirection.UP_EDGE, ViewDirection.RIGHT_EDGE)
            
        # If at bottom of problem, i.e. the solution area,
        # write and move to next column if there is one
        elif last_coord[0] == vertical_view.shape[0] - 1:
            if last_coord[1] > 0:
                next_view_direction = ViewDirection(ViewDirection.UP_EDGE, ViewDirection.LEFT_RELATIVE)
            else:
                next_view_direction = None
            write = str(self.current_value % 10)
            self.current_value = self.current_value // 10
            
        # If at second to last row of problem, i.e. the bar, 
        # do nothing and shift view down
        elif last_coord[0] == vertical_view.shape[0] - 2:
            next_view_direction = ViewDirection(ViewDirection.DOWN_RELATIVE, ViewDirection.DO_NOTHING)
            
        # If at an operand, 
        # add to the current_value, shift view down
        else:
            view_value = vertical_view[last_coord]
            if view_value != ' ':
                self.current_value += int(view_value)
            
            next_view_direction = ViewDirection(ViewDirection.DOWN_RELATIVE, ViewDirection.DO_NOTHING)
            
        self.view_dir_history.append(next_view_direction)
        next_coord = next_view_direction.get_coord(last_coord, vertical_view.shape) if next_view_direction else None
        self.coord_history.append(next_coord)
        self.write_history.append(write)
        return next_view_direction, next_coord, write

In [266]:
e = Expression(Expression.add, [999]*11)
solver = AdditionSolver(e)
print(e.vertical_view_sol)
next_view_direction, next_coord, write = solver.action(None)
print(next_view_direction, next_coord, write)
while (next_view_direction is not None):
    next_view_direction, next_coord, write = solver.action(next_view_direction)
    print(next_view_direction, next_coord, write)

[[' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 [' ' ' ' '9' '9' '9']
 ['_' '_' '_' '_' '_']
 ['1' '0' '9' '8' '9']]
('UP_EDGE', 'RIGHT_EDGE') (0, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (1, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (2, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (3, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (4, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (5, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (6, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (7, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (8, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (9, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (10, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (11, 4) None
('DOWN_RELATIVE', 'DO_NOTHING') (12, 4) None
('UP_EDGE', 'LEFT_RELATIVE') (0, 3) 9
('DOWN_RELATIVE', 'DO_NOTHING') (1, 3) None
('DOWN_RELATIVE', 'DO_NOTHING') (2, 3) None
('DOW